# Amazon Bedrock AgentCore Runtime で Amazon Bedrock モデルを使用した LangGraph エージェントをホスティング

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime を使用して既存のエージェントをホスティングする方法を学びます。

LangGraph と Amazon Bedrock モデルの例に焦点を当てます。Amazon Bedrock モデルを使用した Strands Agents については[こちら](../01-strands-with-bedrock-model)、OpenAI モデルを使用した Strands Agents については[こちら](../03-strands-with-openai-model)をご確認ください。

### チュートリアル詳細

| 情報             | 詳細                                                                          |
|:-----------------|:-----------------------------------------------------------------------------|
| チュートリアルタイプ | 会話形式                                                                     |
| エージェントタイプ  | シングル                                                                      |
| エージェントフレームワーク | LangGraph                                                              |
| LLM モデル        | Anthropic Claude Haiku 4.5                                                    |
| チュートリアル構成  | AgentCore Runtime でのエージェントホスティング。LangGraph と Amazon Bedrock モデルの使用 |
| チュートリアル分野  | クロスバーティカル                                                             |
| 例の複雑さ        | 簡単                                                                          |
| 使用 SDK         | Amazon BedrockAgentCore Python SDK と boto3                                   |

### チュートリアルアーキテクチャ

このチュートリアルでは、既存のエージェントを AgentCore Runtime にデプロイする方法を説明します。

デモンストレーション目的で、Amazon Bedrock モデルを使用した LangGraph エージェントを使用します。

この例では、`get_weather` と `get_time` の2つのツールを持つ非常にシンプルなエージェントを使用します。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントホスティング
* Amazon Bedrock モデルの使用
* LangGraph の使用

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker が実行中

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## エージェントの作成とローカルでの実験

AgentCore Runtime にエージェントをデプロイする前に、実験目的でローカルで開発・実行してみましょう。

本番用エージェントアプリケーションでは、エージェントの作成プロセスと呼び出しプロセスを分離する必要があります。AgentCore Runtime では、エージェントの呼び出し部分を `@app.entrypoint` デコレーターで装飾し、ランタイムのエントリーポイントとして使用します。まず、実験フェーズ中に各エージェントがどのように開発されるかを見てみましょう。

ここでのアーキテクチャは以下のようになります：

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
import argparse
import json
import operator
import math

# 計算機ツールを作成
@tool
def calculator(expression: str) -> str:
    """
    数学的な式の結果を計算します。
    
    Args:
        expression: 文字列としての数学式（例："2 + 3 * 4"、"sqrt(16)"、"sin(pi/2)"）
    
    Returns:
        計算結果を文字列として返す
    """
    try:
        # 式で使用できる安全な関数を定義
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # 数学関数
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # 基本演算子（明示的使用用）
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # 式を安全に評価
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "エラー: ゼロ除算"
    except ValueError as e:
        return f"エラー: 無効な値 - {str(e)}"
    except SyntaxError:
        return "エラー: 無効な数学式"
    except Exception as e:
        return f"エラー: {str(e)}"

# カスタム天気ツールを作成
@tool
def weather():
    """天気を取得"""  # ダミー実装
    return "sunny"

# 手動 LangGraph 構築を使用してエージェントを定義
def create_agent():
    """LangGraph エージェントを作成して設定"""
    from langchain_aws import ChatBedrock
    
    # LLM を初期化（必要に応じてモデルとパラメータを調整）
    llm = ChatBedrock(
        model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # またはお好みのモデル
        model_kwargs={"temperature": 0.1}
    )
    
    # ツールを LLM にバインド
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # システムメッセージ
    system_message = "あなたは親切なアシスタントです。簡単な数学計算ができ、天気を教えることができます。"
    
    # チャットボットノードを定義
    def chatbot(state: MessagesState):
        # システムメッセージがまだ存在しない場合は追加
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # グラフを作成
    graph_builder = StateGraph(MessagesState)
    
    # ノードを追加
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # エッジを追加
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # エントリーポイントを設定
    graph_builder.set_entry_point("chatbot")
    
    # グラフをコンパイル
    return graph_builder.compile()

# エージェントを初期化
agent = create_agent()

def langgraph_bedrock(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    
    # LangGraph が期待する形式で入力を作成
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # 最終メッセージのコンテンツを抽出
    return response["messages"][-1].content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = langgraph_bedrock(json.loads(args.payload))
    print(response)

#### ローカルエージェントの呼び出し

In [ ]:
!python langgraph_bedrock.py '{"prompt": "今の天気はどうですか？"}'

## AgentCore Runtime へのデプロイ用にエージェントを準備

次に、エージェントを AgentCore Runtime にデプロイしましょう。そのためには以下が必要です：
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポート
* `app = BedrockAgentCoreApp()` でコード内で App を初期化
* 呼び出し関数を `@app.entrypoint` デコレーターで装飾
* `app.run()` で AgentCore Runtime にエージェントの実行を制御させる

### Amazon Bedrock モデルを使用した LangGraph
Amazon Bedrock モデルを使用した LangGraph から始めましょう。異なるフレームワークやモデルを使用した他の例は、親ディレクトリで利用可能です。

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import operator
import math

app = BedrockAgentCoreApp()

# 計算機ツールを作成
@tool
def calculator(expression: str) -> str:
    """
    数学的な式の結果を計算します。
    
    Args:
        expression: 文字列としての数学式（例："2 + 3 * 4"、"sqrt(16)"、"sin(pi/2)"）
    
    Returns:
        計算結果を文字列として返す
    """
    try:
        # 式で使用できる安全な関数を定義
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # 数学関数
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # 基本演算子（明示的使用用）
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # 式を安全に評価
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "エラー: ゼロ除算"
    except ValueError as e:
        return f"エラー: 無効な値 - {str(e)}"
    except SyntaxError:
        return "エラー: 無効な数学式"
    except Exception as e:
        return f"エラー: {str(e)}"

# カスタム天気ツールを作成
@tool
def weather():
    """天気を取得"""  # ダミー実装
    return "sunny"

# 手動 LangGraph 構築を使用してエージェントを定義
def create_agent():
    """LangGraph エージェントを作成して設定"""
    from langchain_aws import ChatBedrock
    
    # LLM を初期化（必要に応じてモデルとパラメータを調整）
    llm = ChatBedrock(
        model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # またはお好みのモデル
        model_kwargs={"temperature": 0.1}
    )
    
    # ツールを LLM にバインド
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # システムメッセージ
    system_message = "あなたは親切なアシスタントです。簡単な数学計算ができ、天気を教えることができます。"
    
    # チャットボットノードを定義
    def chatbot(state: MessagesState):
        # システムメッセージがまだ存在しない場合は追加
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # グラフを作成
    graph_builder = StateGraph(MessagesState)
    
    # ノードを追加
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # エッジを追加
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # エントリーポイントを設定
    graph_builder.set_entry_point("chatbot")
    
    # グラフをコンパイル
    return graph_builder.compile()

# エージェントを初期化
agent = create_agent()

@app.entrypoint
def langgraph_bedrock(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    
    # LangGraph が期待する形式で入力を作成
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # 最終メッセージのコンテンツを抽出
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()

## 裏側で何が起きているか？

`BedrockAgentCoreApp` を使用すると、自動的に以下が行われます：

* ポート 8080 でリッスンする HTTP サーバーを作成
* エージェントの要件を処理するために必要な `/invocations` エンドポイントを実装
* ヘルスチェック用の `/ping` エンドポイントを実装（非同期エージェントにとって非常に重要）
* 適切なコンテンツタイプとレスポンス形式を処理
* AWS 標準に従ったエラーハンドリングを管理

## エージェントを AgentCore Runtime にデプロイ

`CreateAgentRuntime` 操作は、コンテナイメージ、環境変数、暗号化設定を指定できる包括的な設定オプションをサポートしています。また、クライアントがエージェントと通信する方法を制御するために、プロトコル設定（HTTP、MCP）と認可メカニズムも設定できます。

**注意：** 運用のベストプラクティスは、コードをコンテナとしてパッケージ化し、CI/CD パイプラインと IaC を使用して ECR にプッシュすることです。

このチュートリアルでは、Amazon Bedrock AgentCore Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイします。

### AgentCore Runtime デプロイの設定

まず、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、および requirements ファイルで AgentCore Runtime デプロイを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Docker ファイルが生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()

agent_name = "langgraph_claude_getting_started"
response = agentcore_runtime.configure(
    entrypoint="langgraph_bedrock.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### エージェントを AgentCore Runtime に起動

Docker ファイルができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtime のステータス確認
AgentCore Runtime をデプロイしたので、デプロイステータスを確認しましょう。

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtime の呼び出し

最後に、ペイロードを使用して AgentCore Runtime を呼び出すことができます。

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "2+2はいくらですか？"})
invoke_response

### 呼び出し結果の処理

呼び出し結果を処理して、アプリケーションに組み込むことができます。

In [ ]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### boto3 を使用した AgentCore Runtime の呼び出し

AgentCore Runtime が作成されたら、任意の AWS SDK で呼び出すことができます。例えば、boto3 の `invoke_agent_runtime` メソッドを使用できます。

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "今の天気はどうですか？"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう。

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# おめでとうございます！